In [ ]:
#clear all
%reset -f

#import packages
import numpy as np
import scipy
import sys
import os
import pandas as pd
import mne
import matplotlib
import h5py
from sklearn.utils import resample
from mne_icalabel import label_components

root = 'F:/Documents/Science/MirRevAdaptEEG'
participants = list(range(0,32))
#specify which erp we are analyzing
erps = 'lrp'
roi = 'medfro'

#pop up plots as separate window & interactive
%matplotlib qt
matplotlib.pyplot.close('all')

In [ ]:
#setting up path/ directory
#access trial type epoched data for each participant
def load_tfr_epochs(pp_num, root_dir, erp_path, task):

    root_directory = root_dir
    data_directory = os.path.join(root_directory, 'data/eeg/')
    id_directory = os.path.join(data_directory, 'p%03d/' % pp_num)
    pp_directory = os.path.join(id_directory, erp_path)
    filename = os.path.join(pp_directory, 'p%03d_%s-epo.fif' % (pp_num, task))

    epochs = mne.read_epochs(filename)

    return epochs, pp_directory

In [ ]:
#set up parameters for time-frequency morlet convolution
def set_tfr_params(l_freq = 6, h_freq = 35, num_freq = 50, cycle = 6, samp_rate = 200):
    freqs = np.logspace(*np.log10([l_freq, h_freq]), num = num_freq)
    n_cycles = cycle
    sfreq = samp_rate
    
    return freqs, n_cycles, sfreq

In [ ]:
#save tfr'd data
def save_tfr_data(pp_num, root_dir, erp_path, data, task, output):
    # Save the tfr'd data
    root_directory = root_dir
    data_directory = os.path.join(root_directory, 'data/eeg/')
    id_directory = os.path.join(data_directory, 'p%03d/' % pp_num)
    pp_directory = os.path.join(id_directory, erp_path)
    out_fname = os.path.join(pp_directory, 'p%03d_%s_%s-tfr.h5' % (pp_num, task, output))
    mne.time_frequency.write_tfrs(out_fname, tfr=data, overwrite = True)

In [ ]:
#load tfr'd data
def load_tfr_data(pp_num, root_dir, erp_path, task, output):
    # Save the tfr'd data
    root_directory = root_dir
    data_directory = os.path.join(root_directory, 'data/eeg/')
    id_directory = os.path.join(data_directory, 'p%03d/' % pp_num)
    pp_directory = os.path.join(id_directory, erp_path)
    out_fname = os.path.join(pp_directory, 'p%03d_%s_%s-tfr.h5' % (pp_num, task, output))
    dat = mne.time_frequency.read_tfrs(out_fname)
    
    return dat

In [ ]:
# After transforming the TFR data into a single signal representative of frequency and channels
# we compare the two signals: same as taking their difference (signal1 minus signal2) and using this in permutation test
def get_clust_perm_test(conditionA, conditionB, pval, n_permutations):
    #take difference of two conditions
    data = np.subtract(conditionA, conditionB)
    np.shape(data)
    #define cluster forming threshold based on p-value
    df = len(participants) - 1
    thresh = scipy.stats.t.ppf(1 - pval / 2, df)
    #run cluster-based permutation test
    T_0, clust_idx, clust_pvals, H0 = mne.stats.permutation_cluster_1samp_test(data, threshold = thresh, 
                                                          n_permutations = n_permutations, tail = 0, 
                                                          adjacency = None, seed = 999, 
                                                          out_type = 'mask', verbose = True)

    return T_0, clust_idx, clust_pvals, H0

In [ ]:
# # save average tfr per participant for each condition
# # morlet wavelet convolution for each participant, according to trial types - POWER

# perturb_conds = ['small_large_aligned', 'smlerrors_rot', 'lrgerrors_rot', 'smlerrors_mir', 'lrgerrors_mir', 'smlerrors_rdm', 'lrgerrors_rdm']

# for pcond in range(0, len(perturb_conds)):
#     for pp in participants:
#         data, ppdir = load_tfr_epochs(pp_num = pp, root_dir = root, erp_path = erps, task = perturb_conds[pcond])
        
#         freqs, n_cycles, sfreq = set_tfr_params()
        
#         #average epochs within participant
#         power = mne.time_frequency.tfr_morlet(
#             data,
#             freqs=freqs,
#             n_cycles=n_cycles,
#             use_fft=False,
#             decim=1,
#             n_jobs=None,
#             return_itc = False,
#             average = True
#         )
        
#         save_tfr_data(pp_num = pp, root_dir = root, erp_path = erps, data = power, task = perturb_conds[pcond], output = "power")

In [ ]:
# load in tfr object, ensure to baseline correct; (tfr shape is channels, freqs, timepts)
# narrow down to channels 
# narrow down to frequencies and timepts
# take mean of theta, alpha, or beta ranges (freqs)
# take mean across electrodes (channels)

perturb_conds = ['small_large_aligned', 'smlerrors_rot', 'lrgerrors_rot', 'smlerrors_mir', 'lrgerrors_mir', 'smlerrors_rdm', 'lrgerrors_rdm']

# medial frontal CHANNELS
channels = ['F1', 'Fz', 'F2',
            'FC1', 'FCz', 'FC2',
            'C1', 'Cz', 'C2']


# BASELINE FOR FEEDBACK ONSET
baseline_t = (-1.3, -1.0)

# THETA
freq_lower = 6
freq_upper = 8

theta_medfro_SmallLargeAligned = []
theta_medfro_SmallRot = []
theta_medfro_LargeRot = []
theta_medfro_SmallMir = []
theta_medfro_LargeMir = []
theta_medfro_SmallRdm = []
theta_medfro_LargeRdm = []

for pcond in range(0, len(perturb_conds)):
    medfroppdat = []
    for pp in participants:
        dat = load_tfr_data(pp_num = pp, root_dir = root, erp_path = erps, task = perturb_conds[pcond], output = "power")
        dat = dat[0]*1e12 #convert V^2 to uV^2
        # apply baseline correction
        dat = dat.apply_baseline(baseline_t, mode='mean')
        # narrow down to channels
        dat = dat.pick_channels(channels)
        #transform to ndarray (channels, freqs, times)
        npdat = dat.data

        # get needed frequency indices
        nfreqs = [] #get indices of frequencies we want
        for i in range(0, len(dat.freqs)):
            if dat.freqs[i] >= freq_lower and dat.freqs[i] <= freq_upper:
                nfreqs.append(i)
        npdat = npdat[:, nfreqs, :]

        # get needed timept indices
        ntimes = list(range(200,401))
        npdat = npdat[:,:,ntimes]
        
        ppdat = np.mean(npdat, axis = 1) #calculate mean across frequencies
        ppdat = np.mean(ppdat, axis = 0) # calculate mean across channels
        
        medfroppdat.append(ppdat)
        
    if pcond == 0:
        theta_medfro_SmallLargeAligned.append(medfroppdat)
    elif pcond == 1:
        theta_medfro_SmallRot.append(medfroppdat)
    elif pcond == 2:
        theta_medfro_LargeRot.append(medfroppdat)
    elif pcond == 3:
        theta_medfro_SmallMir.append(medfroppdat)
    elif pcond == 4:
        theta_medfro_LargeMir.append(medfroppdat)
    elif pcond == 5:
        theta_medfro_SmallRdm.append(medfroppdat)
    elif pcond == 6:
        theta_medfro_LargeRdm.append(medfroppdat)

# ALPHA
freq_lower = 9
freq_upper = 13

alpha_medfro_SmallLargeAligned = []
alpha_medfro_SmallRot = []
alpha_medfro_LargeRot = []
alpha_medfro_SmallMir = []
alpha_medfro_LargeMir = []
alpha_medfro_SmallRdm = []
alpha_medfro_LargeRdm = []

for pcond in range(0, len(perturb_conds)):
    medfroppdat = []
    for pp in participants:
        dat = load_tfr_data(pp_num = pp, root_dir = root, erp_path = erps, task = perturb_conds[pcond], output = "power")
        dat = dat[0]*1e12 #convert V^2 to uV^2
        # apply baseline correction
        dat = dat.apply_baseline(baseline_t, mode='mean')
        # narrow down to channels
        dat = dat.pick_channels(channels)
        #transform to ndarray (channels, freqs, times)
        npdat = dat.data

        # get needed frequency indices
        nfreqs = [] #get indices of frequencies we want
        for i in range(0, len(dat.freqs)):
            if dat.freqs[i] >= freq_lower and dat.freqs[i] <= freq_upper:
                nfreqs.append(i)
        npdat = npdat[:, nfreqs, :]

        # get needed timept indices
        ntimes = list(range(200,401))
        npdat = npdat[:,:,ntimes]
        
        ppdat = np.mean(npdat, axis = 1) #calculate mean across frequencies
        ppdat = np.mean(ppdat, axis = 0) # calculate mean across channels
        
        medfroppdat.append(ppdat)
        
    if pcond == 0:
        alpha_medfro_SmallLargeAligned.append(medfroppdat)
    elif pcond == 1:
        alpha_medfro_SmallRot.append(medfroppdat)
    elif pcond == 2:
        alpha_medfro_LargeRot.append(medfroppdat)
    elif pcond == 3:
        alpha_medfro_SmallMir.append(medfroppdat)
    elif pcond == 4:
        alpha_medfro_LargeMir.append(medfroppdat)
    elif pcond == 5:
        alpha_medfro_SmallRdm.append(medfroppdat)
    elif pcond == 6:
        alpha_medfro_LargeRdm.append(medfroppdat)

# BETA
freq_lower = 13
freq_upper = 25

beta_medfro_SmallLargeAligned = []
beta_medfro_SmallRot = []
beta_medfro_LargeRot = []
beta_medfro_SmallMir = []
beta_medfro_LargeMir = []
beta_medfro_SmallRdm = []
beta_medfro_LargeRdm = []

for pcond in range(0, len(perturb_conds)):
    medfroppdat = []
    for pp in participants:
        dat = load_tfr_data(pp_num = pp, root_dir = root, erp_path = erps, task = perturb_conds[pcond], output = "power")
        dat = dat[0]*1e12 #convert V^2 to uV^2
        # apply baseline correction
        dat = dat.apply_baseline(baseline_t, mode='mean')
        # narrow down to channels
        dat = dat.pick_channels(channels)
        #transform to ndarray (channels, freqs, times)
        npdat = dat.data

        # get needed frequency indices
        nfreqs = [] #get indices of frequencies we want
        for i in range(0, len(dat.freqs)):
            if dat.freqs[i] >= freq_lower and dat.freqs[i] <= freq_upper:
                nfreqs.append(i)
        npdat = npdat[:, nfreqs, :]

        # get needed timept indices
        ntimes = list(range(200,401))
        npdat = npdat[:,:,ntimes]
        
        ppdat = np.mean(npdat, axis = 1) #calculate mean across frequencies
        ppdat = np.mean(ppdat, axis = 0) # calculate mean across channels
        
        medfroppdat.append(ppdat)
        
    if pcond == 0:
        beta_medfro_SmallLargeAligned.append(medfroppdat)
    elif pcond == 1:
        beta_medfro_SmallRot.append(medfroppdat)
    elif pcond == 2:
        beta_medfro_LargeRot.append(medfroppdat)
    elif pcond == 3:
        beta_medfro_SmallMir.append(medfroppdat)
    elif pcond == 4:
        beta_medfro_LargeMir.append(medfroppdat)
    elif pcond == 5:
        beta_medfro_SmallRdm.append(medfroppdat)
    elif pcond == 6:
        beta_medfro_LargeRdm.append(medfroppdat)
    

In [ ]:
# Get timepts to use for indices given by cluster-based permutation later and for timepts in saved data frames
# But only grab -1.5 to 0 sec time-locked to feedback onset (idx= 100:401)
root_directory = root
pp = 0 #only need one participant

# we can use aligned data
dat = load_tfr_data(pp_num = pp, root_dir = root, erp_path = erps, task = 'small_large_aligned', output = "power")
time = dat[0].times
time = time[200:401] #get only timepoints we want

In [ ]:
# save tfr data across participants in each condition (used for plotting signals)
theta_flists = [theta_medfro_SmallLargeAligned, theta_medfro_SmallRot, theta_medfro_LargeRot, 
               theta_medfro_SmallRdm, theta_medfro_LargeRdm,
               theta_medfro_SmallMir, theta_medfro_LargeMir]
alpha_flists = [alpha_medfro_SmallLargeAligned, alpha_medfro_SmallRot, alpha_medfro_LargeRot, 
                alpha_medfro_SmallRdm, alpha_medfro_LargeRdm, 
                alpha_medfro_SmallMir, alpha_medfro_LargeMir]
beta_flists = [beta_medfro_SmallLargeAligned, beta_medfro_SmallRot, beta_medfro_LargeRot, 
               beta_medfro_SmallRdm, beta_medfro_LargeRdm,
               beta_medfro_SmallMir, beta_medfro_LargeMir]

theta_conditionnames = ['theta_smalllarge_aligned', 'theta_smallrot', 'theta_largerot', 'theta_smallrdm', 'theta_largerdm', 'theta_smallmir', 'theta_largemir']
alpha_conditionnames = ['alpha_smalllarge_aligned', 'alpha_smallrot', 'alpha_largerot', 'alpha_smallrdm', 'alpha_largerdm', 'alpha_smallmir', 'alpha_largemir']
beta_conditionnames = ['beta_smalllarge_aligned', 'beta_smallrot', 'beta_largerot', 'beta_smallrdm', 'beta_largerdm', 'beta_smallmir', 'beta_largemir']

freq_bands = ['theta', 'alpha', 'beta']

for band in range(0, len(freq_bands)):
    if band == 0:
        for f in range(0, len(theta_flists)):
            df = pd.DataFrame(theta_flists[f][0]).T
            df['time'] = time

            filename = os.path.join('F:/Documents/Science/MirRevAdaptEEG/data/', 'TFR_medfro_%s_%s.csv' % (theta_conditionnames[f], erps))
            df.to_csv(filename)
    elif band == 1:
        for f in range(0, len(alpha_flists)):
            df = pd.DataFrame(alpha_flists[f][0]).T
            df['time'] = time

            filename = os.path.join('F:/Documents/Science/MirRevAdaptEEG/data/', 'TFR_medfro_%s_%s.csv' % (alpha_conditionnames[f], erps))
            df.to_csv(filename)
    elif band == 2:
        for f in range(0, len(beta_flists)):
            df = pd.DataFrame(beta_flists[f][0]).T
            df['time'] = time

            filename = os.path.join('F:/Documents/Science/MirRevAdaptEEG/data/', 'TFR_medfro_%s_%s.csv' % (beta_conditionnames[f], erps))
            df.to_csv(filename)
            

In [ ]:
# First, we can compare each condition to aligned
# Generate a data frame to tabulate condition, cluster indices, cluster timepts, p values
# This information can then be included in plots

theta_flists = [theta_medfro_SmallRot, theta_medfro_LargeRot, 
               theta_medfro_SmallRdm, theta_medfro_LargeRdm,
               theta_medfro_SmallMir, theta_medfro_LargeMir]
alpha_flists = [alpha_medfro_SmallRot, alpha_medfro_LargeRot, 
                alpha_medfro_SmallRdm, alpha_medfro_LargeRdm, 
                alpha_medfro_SmallMir, alpha_medfro_LargeMir]
beta_flists = [beta_medfro_SmallRot, beta_medfro_LargeRot, 
               beta_medfro_SmallRdm, beta_medfro_LargeRdm,
               beta_medfro_SmallMir, beta_medfro_LargeMir]

theta_conditionnames = ['theta_smallrot', 'theta_largerot', 'theta_smallrdm', 'theta_largerdm', 'theta_smallmir', 'theta_largemir']
alpha_conditionnames = ['alpha_smallrot', 'alpha_largerot', 'alpha_smallrdm', 'alpha_largerdm', 'alpha_smallmir', 'alpha_largemir']
beta_conditionnames = ['beta_smallrot', 'beta_largerot', 'beta_smallrdm', 'beta_largerdm', 'beta_smallmir', 'beta_largemir']

freq_bands = ['theta', 'alpha', 'beta']
p = 0.05
perms = 1000

condition = []
clust_idx_start = []
clust_idx_end = []
time_start = []
time_end = []
p_values = []

for band in range(0, len(freq_bands)):
    if band == 0:
        for f in range(0, len(theta_flists)):
            T_0, clust_idx, clust_pvals, H0 = get_clust_perm_test(theta_flists[f][0], theta_medfro_SmallLargeAligned[0], p, perms)
#             print(clust_idx, clust_pvals)
            if len(clust_idx) == 0:
                condition.append(theta_conditionnames[f])
                clust_idx_start.append(np.nan)
                clust_idx_end.append(np.nan)
                time_start.append(np.nan)
                time_end.append(np.nan)
                p_values.append(np.nan)
            else:
                for clust in range(0, len(clust_idx)):
                    cluster = clust_idx[clust][0] #to get the slice sequence we need
        
                    cluster_start = cluster.start
                    clust_idx_start.append(cluster_start)
        
                    cluster_end = cluster.stop
                    clust_idx_end.append(cluster_end)
        
                    time_idx_start = time[cluster_start]
                    time_start.append(time_idx_start)
        
                    time_idx_end = time[cluster_end - 1] #minus one because python indexing does not include ending value
                    time_end.append(time_idx_end)
        
                    clust_p = clust_pvals[clust]
                    p_values.append(clust_p)
        
                    condition.append(theta_conditionnames[f])
            
    elif band == 1:
        for f in range(0, len(alpha_flists)):
            T_0, clust_idx, clust_pvals, H0 = get_clust_perm_test(alpha_flists[f][0], alpha_medfro_SmallLargeAligned[0], p, perms)
#             print(clust_idx, clust_pvals)
            if len(clust_idx) == 0:
                condition.append(alpha_conditionnames[f])
                clust_idx_start.append(np.nan)
                clust_idx_end.append(np.nan)
                time_start.append(np.nan)
                time_end.append(np.nan)
                p_values.append(np.nan)
            else:
                for clust in range(0, len(clust_idx)):
                    cluster = clust_idx[clust][0] #to get the slice sequence we need
        
                    cluster_start = cluster.start
                    clust_idx_start.append(cluster_start)
        
                    cluster_end = cluster.stop
                    clust_idx_end.append(cluster_end)
        
                    time_idx_start = time[cluster_start]
                    time_start.append(time_idx_start)
        
                    time_idx_end = time[cluster_end - 1] #minus one because python indexing does not include ending value
                    time_end.append(time_idx_end)
        
                    clust_p = clust_pvals[clust]
                    p_values.append(clust_p)
        
                    condition.append(alpha_conditionnames[f])
    elif band == 2:
        for f in range(0, len(beta_flists)):
            T_0, clust_idx, clust_pvals, H0 = get_clust_perm_test(beta_flists[f][0], beta_medfro_SmallLargeAligned[0], p, perms)
#             print(clust_idx, clust_pvals)
            if len(clust_idx) == 0:
                condition.append(beta_conditionnames[f])
                clust_idx_start.append(np.nan)
                clust_idx_end.append(np.nan)
                time_start.append(np.nan)
                time_end.append(np.nan)
                p_values.append(np.nan)
            else:
                for clust in range(0, len(clust_idx)):
                    cluster = clust_idx[clust][0] #to get the slice sequence we need
        
                    cluster_start = cluster.start
                    clust_idx_start.append(cluster_start)
        
                    cluster_end = cluster.stop
                    clust_idx_end.append(cluster_end)
        
                    time_idx_start = time[cluster_start]
                    time_start.append(time_idx_start)
        
                    time_idx_end = time[cluster_end - 1] #minus one because python indexing does not include ending value
                    time_end.append(time_idx_end)
        
                    clust_p = clust_pvals[clust]
                    p_values.append(clust_p)
        
                    condition.append(beta_conditionnames[f])
        
perm_test = pd.DataFrame(
    {'condition': condition,
        'clust_idx_start': clust_idx_start,
        'clust_idx_end': clust_idx_end,
        'time_start': time_start,
        'time_end': time_end,
        'p_values': p_values})

perm_test_filename = os.path.join('F:/Documents/Science/MirRevAdaptEEG/data/', 'TFR_Permutation_test_ErrorSize_vsAligned_%s_%s.csv' % (erps, roi))
perm_test.to_csv(perm_test_filename)


In [ ]:
# Next, we subtract aligned from each condition, so that we can compare small vs large in each perturbation type

freq_bands = ['theta', 'alpha', 'beta']
diffconds = ['smallrot', 'largerot', 'smallrdm', 'largerdm', 'smallmir', 'largemir']

theta_medfro_smallrot_diff = []
theta_medfro_largerot_diff = []
theta_medfro_smallrdm_diff = []
theta_medfro_largerdm_diff = []
theta_medfro_smallmir_diff = []
theta_medfro_largemir_diff = []

alpha_medfro_smallrot_diff = []
alpha_medfro_largerot_diff = []
alpha_medfro_smallrdm_diff = []
alpha_medfro_largerdm_diff = []
alpha_medfro_smallmir_diff = []
alpha_medfro_largemir_diff = []

beta_medfro_smallrot_diff = []
beta_medfro_largerot_diff = []
beta_medfro_smallrdm_diff = []
beta_medfro_largerdm_diff = []
beta_medfro_smallmir_diff = []
beta_medfro_largemir_diff = []

for band in range(0, len(freq_bands)):
    if band == 0:
        for cond in range(0, len(diffconds)):
            if cond == 0:
                diffevks = np.subtract(theta_medfro_SmallRot[0], theta_medfro_SmallLargeAligned[0])
                theta_medfro_smallrot_diff.append(diffevks)
                theta_medfro_smallrot_diff = theta_medfro_smallrot_diff[0] #to keep shape of object consistent
            elif cond == 1:
                diffevks = np.subtract(theta_medfro_LargeRot[0], theta_medfro_SmallLargeAligned[0])
                theta_medfro_largerot_diff.append(diffevks)
                theta_medfro_largerot_diff = theta_medfro_largerot_diff[0]
            elif cond == 2:
                diffevks = np.subtract(theta_medfro_SmallRdm[0], theta_medfro_SmallLargeAligned[0])
                theta_medfro_smallrdm_diff.append(diffevks)
                theta_medfro_smallrdm_diff = theta_medfro_smallrdm_diff[0]
            elif cond == 3:
                diffevks = np.subtract(theta_medfro_LargeRdm[0], theta_medfro_SmallLargeAligned[0])
                theta_medfro_largerdm_diff.append(diffevks)
                theta_medfro_largerdm_diff = theta_medfro_largerdm_diff[0]
            elif cond == 4:
                diffevks = np.subtract(theta_medfro_SmallMir[0], theta_medfro_SmallLargeAligned[0])
                theta_medfro_smallmir_diff.append(diffevks)
                theta_medfro_smallmir_diff = theta_medfro_smallmir_diff[0]
            elif cond == 5:
                diffevks = np.subtract(theta_medfro_LargeMir[0], theta_medfro_SmallLargeAligned[0])
                theta_medfro_largemir_diff.append(diffevks)
                theta_medfro_largemir_diff = theta_medfro_largemir_diff[0]
    elif band == 1:
        for cond in range(0, len(diffconds)):
            if cond == 0:
                diffevks = np.subtract(alpha_medfro_SmallRot[0], alpha_medfro_SmallLargeAligned[0])
                alpha_medfro_smallrot_diff.append(diffevks)
                alpha_medfro_smallrot_diff = alpha_medfro_smallrot_diff[0] #to keep shape of object consistent
            elif cond == 1:
                diffevks = np.subtract(alpha_medfro_LargeRot[0], alpha_medfro_SmallLargeAligned[0])
                alpha_medfro_largerot_diff.append(diffevks)
                alpha_medfro_largerot_diff = alpha_medfro_largerot_diff[0]
            elif cond == 2:
                diffevks = np.subtract(alpha_medfro_SmallRdm[0], alpha_medfro_SmallLargeAligned[0])
                alpha_medfro_smallrdm_diff.append(diffevks)
                alpha_medfro_smallrdm_diff = alpha_medfro_smallrdm_diff[0]
            elif cond == 3:
                diffevks = np.subtract(alpha_medfro_LargeRdm[0], alpha_medfro_SmallLargeAligned[0])
                alpha_medfro_largerdm_diff.append(diffevks)
                alpha_medfro_largerdm_diff = alpha_medfro_largerdm_diff[0]
            elif cond == 4:
                diffevks = np.subtract(alpha_medfro_SmallMir[0], alpha_medfro_SmallLargeAligned[0])
                alpha_medfro_smallmir_diff.append(diffevks)
                alpha_medfro_smallmir_diff = alpha_medfro_smallmir_diff[0]
            elif cond == 5:
                diffevks = np.subtract(alpha_medfro_LargeMir[0], alpha_medfro_SmallLargeAligned[0])
                alpha_medfro_largemir_diff.append(diffevks)
                alpha_medfro_largemir_diff = alpha_medfro_largemir_diff[0]
    elif band == 2:
        for cond in range(0, len(diffconds)):
            if cond == 0:
                diffevks = np.subtract(beta_medfro_SmallRot[0], beta_medfro_SmallLargeAligned[0])
                beta_medfro_smallrot_diff.append(diffevks)
                beta_medfro_smallrot_diff = beta_medfro_smallrot_diff[0] #to keep shape of object consistent
            elif cond == 1:
                diffevks = np.subtract(beta_medfro_LargeRot[0], beta_medfro_SmallLargeAligned[0])
                beta_medfro_largerot_diff.append(diffevks)
                beta_medfro_largerot_diff = beta_medfro_largerot_diff[0]
            elif cond == 2:
                diffevks = np.subtract(beta_medfro_SmallRdm[0], beta_medfro_SmallLargeAligned[0])
                beta_medfro_smallrdm_diff.append(diffevks)
                beta_medfro_smallrdm_diff = beta_medfro_smallrdm_diff[0]
            elif cond == 3:
                diffevks = np.subtract(beta_medfro_LargeRdm[0], beta_medfro_SmallLargeAligned[0])
                beta_medfro_largerdm_diff.append(diffevks)
                beta_medfro_largerdm_diff = beta_medfro_largerdm_diff[0]
            elif cond == 4:
                diffevks = np.subtract(beta_medfro_SmallMir[0], beta_medfro_SmallLargeAligned[0])
                beta_medfro_smallmir_diff.append(diffevks)
                beta_medfro_smallmir_diff = beta_medfro_smallmir_diff[0]
            elif cond == 5:
                diffevks = np.subtract(beta_medfro_LargeMir[0], beta_medfro_SmallLargeAligned[0])
                beta_medfro_largemir_diff.append(diffevks)
                beta_medfro_largemir_diff = beta_medfro_largemir_diff[0]

In [ ]:
# save aligned (baseline) subtracted tfr data across participants in each condition (used for plotting signals)
theta_flists = [theta_medfro_smallrot_diff, theta_medfro_largerot_diff, 
                theta_medfro_smallrdm_diff, theta_medfro_largerdm_diff, 
                theta_medfro_smallmir_diff, theta_medfro_largemir_diff]
alpha_flists = [alpha_medfro_smallrot_diff, alpha_medfro_largerot_diff, 
                alpha_medfro_smallrdm_diff, alpha_medfro_largerdm_diff, 
                alpha_medfro_smallmir_diff, alpha_medfro_largemir_diff]
beta_flists = [beta_medfro_smallrot_diff, beta_medfro_largerot_diff, 
               beta_medfro_smallrdm_diff, beta_medfro_largerdm_diff, 
               beta_medfro_smallmir_diff, beta_medfro_largemir_diff]

theta_conditionnames = ['theta_smallrot', 'theta_largerot', 'theta_smallrdm', 'theta_largerdm', 'theta_smallmir', 'theta_largemir']
alpha_conditionnames = ['alpha_smallrot', 'alpha_largerot', 'alpha_smallrdm', 'alpha_largerdm', 'alpha_smallmir', 'alpha_largemir']
beta_conditionnames = ['beta_smallrot', 'beta_largerot', 'beta_smallrdm', 'beta_largerdm', 'beta_smallmir', 'beta_largemir']

freq_bands = ['theta', 'alpha', 'beta']

for band in range(0, len(freq_bands)):
    if band == 0:
        for f in range(0, len(theta_flists)):
            df = pd.DataFrame(theta_flists[f]).T #no need to index [0]
            df['time'] = time

            filename = os.path.join('F:/Documents/Science/MirRevAdaptEEG/data/', 'TFR_medfro_DiffWaves_%s_%s.csv' % (theta_conditionnames[f], erps))
            df.to_csv(filename)
    elif band == 1:
        for f in range(0, len(alpha_flists)):
            df = pd.DataFrame(alpha_flists[f]).T #no need to index [0]
            df['time'] = time

            filename = os.path.join('F:/Documents/Science/MirRevAdaptEEG/data/', 'TFR_medfro_DiffWaves_%s_%s.csv' % (alpha_conditionnames[f], erps))
            df.to_csv(filename)
    elif band == 2:
        for f in range(0, len(beta_flists)):
            df = pd.DataFrame(beta_flists[f]).T
            df['time'] = time

            filename = os.path.join('F:/Documents/Science/MirRevAdaptEEG/data/', 'TFR_medfro_DiffWaves_%s_%s.csv' % (beta_conditionnames[f], erps))
            df.to_csv(filename)

In [ ]:
# Compare small VS large for each perturbation
# Generate a data frame to tabulate condition, cluster indices, cluster timepts, p values
# This information can then be included in plots
p = 0.05
perms = 1000

condition = []
clust_idx_start = []
clust_idx_end = []
time_start = []
time_end = []
p_values = []

freq_bands = ['theta', 'alpha', 'beta']

for band in range(0, len(freq_bands)):
    if band == 0:
        theta_conditionnames = ['theta_medfro_rot', 'theta_medfro_rdm', 'theta_medfro_mir']
        for c in range(0, len(theta_conditionnames)):
            if c == 0:
                T_0, clust_idx, clust_pvals, H0 = get_clust_perm_test(theta_medfro_largerot_diff, theta_medfro_smallrot_diff, p, perms)
        #         print(clust_idx, clust_pvals)
                if len(clust_idx) == 0:
                    condition.append(theta_conditionnames[c])
                    clust_idx_start.append(np.nan)
                    clust_idx_end.append(np.nan)
                    time_start.append(np.nan)
                    time_end.append(np.nan)
                    p_values.append(np.nan)
                else:
                    for clust in range(0, len(clust_idx)):
                        cluster = clust_idx[clust][0] #to get the slice sequence we need
        
                        cluster_start = cluster.start
                        clust_idx_start.append(cluster_start)
        
                        cluster_end = cluster.stop
                        clust_idx_end.append(cluster_end)
        
                        time_idx_start = time[cluster_start]
                        time_start.append(time_idx_start)
        
                        time_idx_end = time[cluster_end - 1] #minus one because python indexing does not include ending value
                        time_end.append(time_idx_end)
        
                        clust_p = clust_pvals[clust]
                        p_values.append(clust_p)
        
                        condition.append(theta_conditionnames[c])
            
            elif c == 1:
                T_0, clust_idx, clust_pvals, H0 = get_clust_perm_test(theta_medfro_largerdm_diff, theta_medfro_smallrdm_diff, p, perms)
                if len(clust_idx) == 0:
                    condition.append(theta_conditionnames[c])
                    clust_idx_start.append(np.nan)
                    clust_idx_end.append(np.nan)
                    time_start.append(np.nan)
                    time_end.append(np.nan)
                    p_values.append(np.nan)
                else:    
                    for clust in range(0, len(clust_idx)):
                        cluster = clust_idx[clust][0] #to get the slice sequence we need
        
                        cluster_start = cluster.start
                        clust_idx_start.append(cluster_start)
        
                        cluster_end = cluster.stop
                        clust_idx_end.append(cluster_end)
        
                        time_idx_start = time[cluster_start]
                        time_start.append(time_idx_start)
        
                        time_idx_end = time[cluster_end - 1] #minus one because python indexing does not include ending value
                        time_end.append(time_idx_end)
        
                        clust_p = clust_pvals[clust]
                        p_values.append(clust_p)
        
                        condition.append(theta_conditionnames[c])
            
            elif c == 2:
                T_0, clust_idx, clust_pvals, H0 = get_clust_perm_test(theta_medfro_largemir_diff, theta_medfro_smallmir_diff, p, perms)
                if len(clust_idx) == 0:
                    condition.append(theta_conditionnames[c])
                    clust_idx_start.append(np.nan)
                    clust_idx_end.append(np.nan)
                    time_start.append(np.nan)
                    time_end.append(np.nan)
                    p_values.append(np.nan)
                else:    
                    for clust in range(0, len(clust_idx)):
                        cluster = clust_idx[clust][0] #to get the slice sequence we need
        
                        cluster_start = cluster.start
                        clust_idx_start.append(cluster_start)
        
                        cluster_end = cluster.stop
                        clust_idx_end.append(cluster_end)
        
                        time_idx_start = time[cluster_start]
                        time_start.append(time_idx_start)
        
                        time_idx_end = time[cluster_end - 1] #minus one because python indexing does not include ending value
                        time_end.append(time_idx_end)
        
                        clust_p = clust_pvals[clust]
                        p_values.append(clust_p)
        
                        condition.append(theta_conditionnames[c])
    elif band == 1:
        alpha_conditionnames = ['alpha_medfro_rot', 'alpha_medfro_rdm', 'alpha_medfro_mir']
        for c in range(0, len(alpha_conditionnames)):
            if c == 0:
                T_0, clust_idx, clust_pvals, H0 = get_clust_perm_test(alpha_medfro_largerot_diff, alpha_medfro_smallrot_diff, p, perms)
        #         print(clust_idx, clust_pvals)
                if len(clust_idx) == 0:
                    condition.append(alpha_conditionnames[c])
                    clust_idx_start.append(np.nan)
                    clust_idx_end.append(np.nan)
                    time_start.append(np.nan)
                    time_end.append(np.nan)
                    p_values.append(np.nan)
                else:
                    for clust in range(0, len(clust_idx)):
                        cluster = clust_idx[clust][0] #to get the slice sequence we need
        
                        cluster_start = cluster.start
                        clust_idx_start.append(cluster_start)
        
                        cluster_end = cluster.stop
                        clust_idx_end.append(cluster_end)
        
                        time_idx_start = time[cluster_start]
                        time_start.append(time_idx_start)
        
                        time_idx_end = time[cluster_end - 1] #minus one because python indexing does not include ending value
                        time_end.append(time_idx_end)
        
                        clust_p = clust_pvals[clust]
                        p_values.append(clust_p)
        
                        condition.append(alpha_conditionnames[c])
            
            elif c == 1:
                T_0, clust_idx, clust_pvals, H0 = get_clust_perm_test(alpha_medfro_largerdm_diff, alpha_medfro_smallrdm_diff, p, perms)
                if len(clust_idx) == 0:
                    condition.append(alpha_conditionnames[c])
                    clust_idx_start.append(np.nan)
                    clust_idx_end.append(np.nan)
                    time_start.append(np.nan)
                    time_end.append(np.nan)
                    p_values.append(np.nan)
                else:    
                    for clust in range(0, len(clust_idx)):
                        cluster = clust_idx[clust][0] #to get the slice sequence we need
        
                        cluster_start = cluster.start
                        clust_idx_start.append(cluster_start)
        
                        cluster_end = cluster.stop
                        clust_idx_end.append(cluster_end)
        
                        time_idx_start = time[cluster_start]
                        time_start.append(time_idx_start)
        
                        time_idx_end = time[cluster_end - 1] #minus one because python indexing does not include ending value
                        time_end.append(time_idx_end)
        
                        clust_p = clust_pvals[clust]
                        p_values.append(clust_p)
        
                        condition.append(alpha_conditionnames[c])
            
            elif c == 2:
                T_0, clust_idx, clust_pvals, H0 = get_clust_perm_test(alpha_medfro_largemir_diff, alpha_medfro_smallmir_diff, p, perms)
                if len(clust_idx) == 0:
                    condition.append(alpha_conditionnames[c])
                    clust_idx_start.append(np.nan)
                    clust_idx_end.append(np.nan)
                    time_start.append(np.nan)
                    time_end.append(np.nan)
                    p_values.append(np.nan)
                else:    
                    for clust in range(0, len(clust_idx)):
                        cluster = clust_idx[clust][0] #to get the slice sequence we need
        
                        cluster_start = cluster.start
                        clust_idx_start.append(cluster_start)
        
                        cluster_end = cluster.stop
                        clust_idx_end.append(cluster_end)
        
                        time_idx_start = time[cluster_start]
                        time_start.append(time_idx_start)
        
                        time_idx_end = time[cluster_end - 1] #minus one because python indexing does not include ending value
                        time_end.append(time_idx_end)
        
                        clust_p = clust_pvals[clust]
                        p_values.append(clust_p)
        
                        condition.append(alpha_conditionnames[c])
            
    elif band == 2:
        beta_conditionnames = ['beta_medfro_rot', 'beta_medfro_rdm', 'beta_medfro_mir']
        for c in range(0, len(beta_conditionnames)):
            if c == 0:
                T_0, clust_idx, clust_pvals, H0 = get_clust_perm_test(beta_medfro_largerot_diff, beta_medfro_smallrot_diff, p, perms)
        #         print(clust_idx, clust_pvals)
                if len(clust_idx) == 0:
                    condition.append(beta_conditionnames[c])
                    clust_idx_start.append(np.nan)
                    clust_idx_end.append(np.nan)
                    time_start.append(np.nan)
                    time_end.append(np.nan)
                    p_values.append(np.nan)
                else:
                    for clust in range(0, len(clust_idx)):
                        cluster = clust_idx[clust][0] #to get the slice sequence we need
        
                        cluster_start = cluster.start
                        clust_idx_start.append(cluster_start)
        
                        cluster_end = cluster.stop
                        clust_idx_end.append(cluster_end)
        
                        time_idx_start = time[cluster_start]
                        time_start.append(time_idx_start)
        
                        time_idx_end = time[cluster_end - 1] #minus one because python indexing does not include ending value
                        time_end.append(time_idx_end)
        
                        clust_p = clust_pvals[clust]
                        p_values.append(clust_p)
        
                        condition.append(beta_conditionnames[c])
            
            elif c == 1:
                T_0, clust_idx, clust_pvals, H0 = get_clust_perm_test(beta_medfro_largerdm_diff, beta_medfro_smallrdm_diff, p, perms)
                if len(clust_idx) == 0:
                    condition.append(beta_conditionnames[c])
                    clust_idx_start.append(np.nan)
                    clust_idx_end.append(np.nan)
                    time_start.append(np.nan)
                    time_end.append(np.nan)
                    p_values.append(np.nan)
                else:    
                    for clust in range(0, len(clust_idx)):
                        cluster = clust_idx[clust][0] #to get the slice sequence we need
        
                        cluster_start = cluster.start
                        clust_idx_start.append(cluster_start)
        
                        cluster_end = cluster.stop
                        clust_idx_end.append(cluster_end)
        
                        time_idx_start = time[cluster_start]
                        time_start.append(time_idx_start)
        
                        time_idx_end = time[cluster_end - 1] #minus one because python indexing does not include ending value
                        time_end.append(time_idx_end)
        
                        clust_p = clust_pvals[clust]
                        p_values.append(clust_p)
        
                        condition.append(beta_conditionnames[c])
            
            elif c == 2:
                T_0, clust_idx, clust_pvals, H0 = get_clust_perm_test(beta_medfro_largemir_diff, beta_medfro_smallmir_diff, p, perms)
                if len(clust_idx) == 0:
                    condition.append(beta_conditionnames[c])
                    clust_idx_start.append(np.nan)
                    clust_idx_end.append(np.nan)
                    time_start.append(np.nan)
                    time_end.append(np.nan)
                    p_values.append(np.nan)
                else:    
                    for clust in range(0, len(clust_idx)):
                        cluster = clust_idx[clust][0] #to get the slice sequence we need
        
                        cluster_start = cluster.start
                        clust_idx_start.append(cluster_start)
        
                        cluster_end = cluster.stop
                        clust_idx_end.append(cluster_end)
        
                        time_idx_start = time[cluster_start]
                        time_start.append(time_idx_start)
        
                        time_idx_end = time[cluster_end - 1] #minus one because python indexing does not include ending value
                        time_end.append(time_idx_end)
        
                        clust_p = clust_pvals[clust]
                        p_values.append(clust_p)
        
                        condition.append(beta_conditionnames[c])
        
perm_test = pd.DataFrame(
    {'condition': condition,
     'clust_idx_start': clust_idx_start,
     'clust_idx_end': clust_idx_end,
     'time_start': time_start,
     'time_end': time_end,
     'p_values': p_values})

perm_test_filename = os.path.join('F:/Documents/Science/MirRevAdaptEEG/data/', 'TFR_Permutation_test_SmallvsLarge_%s_%s.csv' % (erps, roi))
perm_test.to_csv(perm_test_filename)

In [ ]:
# Next step is to subtract small from large condition, to generate a single signal for each perturbation
diffconds = ['rot', 'rdm', 'mir']
freq_bands = ['theta', 'alpha', 'beta']

theta_medfro_rot_diff = []
theta_medfro_rdm_diff = []
theta_medfro_mir_diff = []

alpha_medfro_rot_diff = []
alpha_medfro_rdm_diff = []
alpha_medfro_mir_diff = []

beta_medfro_rot_diff = []
beta_medfro_rdm_diff = []
beta_medfro_mir_diff = []

for band in range(0, len(freq_bands)):
    if band == 0:
        for cond in range(0, len(diffconds)):
            if cond == 0:
                diffevks = np.subtract(theta_medfro_largerot_diff, theta_medfro_smallrot_diff)
                theta_medfro_rot_diff.append(diffevks)
                theta_medfro_rot_diff = theta_medfro_rot_diff[0] #to keep shape of object consistent
            elif cond == 1:
                diffevks = np.subtract(theta_medfro_largerdm_diff, theta_medfro_smallrdm_diff)
                theta_medfro_rdm_diff.append(diffevks)
                theta_medfro_rdm_diff = theta_medfro_rdm_diff[0]
            elif cond == 2:
                diffevks = np.subtract(theta_medfro_largemir_diff, theta_medfro_smallmir_diff)
                theta_medfro_mir_diff.append(diffevks)
                theta_medfro_mir_diff = theta_medfro_mir_diff[0]
    elif band == 1:
        for cond in range(0, len(diffconds)):
            if cond == 0:
                diffevks = np.subtract(alpha_medfro_largerot_diff, alpha_medfro_smallrot_diff)
                alpha_medfro_rot_diff.append(diffevks)
                alpha_medfro_rot_diff = alpha_medfro_rot_diff[0] #to keep shape of object consistent
            elif cond == 1:
                diffevks = np.subtract(alpha_medfro_largerdm_diff, alpha_medfro_smallrdm_diff)
                alpha_medfro_rdm_diff.append(diffevks)
                alpha_medfro_rdm_diff = alpha_medfro_rdm_diff[0]
            elif cond == 2:
                diffevks = np.subtract(alpha_medfro_largemir_diff, alpha_medfro_smallmir_diff)
                alpha_medfro_mir_diff.append(diffevks)
                alpha_medfro_mir_diff = alpha_medfro_mir_diff[0]
    elif band == 2:
        for cond in range(0, len(diffconds)):
            if cond == 0:
                diffevks = np.subtract(beta_medfro_largerot_diff, beta_medfro_smallrot_diff)
                beta_medfro_rot_diff.append(diffevks)
                beta_medfro_rot_diff = beta_medfro_rot_diff[0] #to keep shape of object consistent
            elif cond == 1:
                diffevks = np.subtract(beta_medfro_largerdm_diff, beta_medfro_smallrdm_diff)
                beta_medfro_rdm_diff.append(diffevks)
                beta_medfro_rdm_diff = beta_medfro_rdm_diff[0]
            elif cond == 2:
                diffevks = np.subtract(beta_medfro_largemir_diff, beta_medfro_smallmir_diff)
                beta_medfro_mir_diff.append(diffevks)
                beta_medfro_mir_diff = beta_medfro_mir_diff[0]

In [ ]:
# save small and large subtracted tfr data across participants in each condition (used for plotting signals)
theta_flists = [theta_medfro_rot_diff, theta_medfro_rdm_diff, theta_medfro_mir_diff]
alpha_flists = [alpha_medfro_rot_diff, alpha_medfro_rdm_diff, alpha_medfro_mir_diff]
beta_flists = [beta_medfro_rot_diff, beta_medfro_rdm_diff, beta_medfro_mir_diff]

theta_conditionnames = ['theta_rot', 'theta_rdm', 'theta_mir']
alpha_conditionnames = ['alpha_rot', 'alpha_rdm', 'alpha_mir']
beta_conditionnames = ['beta_rot', 'beta_rdm', 'beta_mir']

freq_bands = ['theta', 'alpha', 'beta']

for band in range(0, len(freq_bands)):
    if band == 0:
        for f in range(0, len(theta_flists)):
            df = pd.DataFrame(theta_flists[f]).T #no need to index [0]
            df['time'] = time

            filename = os.path.join('F:/Documents/Science/MirRevAdaptEEG/data/', 'TFR_medfro_DiffWaves_SvL_%s_%s.csv' % (theta_conditionnames[f], erps))
            df.to_csv(filename)
    elif band == 1:
        for f in range(0, len(alpha_flists)):
            df = pd.DataFrame(alpha_flists[f]).T #no need to index [0]
            df['time'] = time

            filename = os.path.join('F:/Documents/Science/MirRevAdaptEEG/data/', 'TFR_medfro_DiffWaves_SvL_%s_%s.csv' % (alpha_conditionnames[f], erps))
            df.to_csv(filename)
    elif band == 2:
        for f in range(0, len(beta_flists)):
            df = pd.DataFrame(beta_flists[f]).T
            df['time'] = time

            filename = os.path.join('F:/Documents/Science/MirRevAdaptEEG/data/', 'TFR_medfro_DiffWaves_SvL_%s_%s.csv' % (beta_conditionnames[f], erps))
            df.to_csv(filename)

In [ ]:
# Compare SMALL vs LARGE for each perturbation
# Generate a data frame to tabulate condition, cluster indices, cluster timepts, p values
# This information can then be included in plots
p = 0.05
perms = 1000

condition = []
clust_idx_start = []
clust_idx_end = []
time_start = []
time_end = []
p_values = []

freq_bands = ['theta', 'alpha', 'beta']

for band in range(0, len(freq_bands)):
    if band == 0:
        theta_conditionnames = ['theta_medfro_rotvmir', 'theta_medfro_rotvrdm', 'theta_medfro_mirvrdm']
        for c in range(0, len(theta_conditionnames)):
            if c == 0:
                T_0, clust_idx, clust_pvals, H0 = get_clust_perm_test(theta_medfro_rot_diff, theta_medfro_mir_diff, p, perms)
        #         print(clust_idx, clust_pvals)
                if len(clust_idx) == 0:
                    condition.append(theta_conditionnames[c])
                    clust_idx_start.append(np.nan)
                    clust_idx_end.append(np.nan)
                    time_start.append(np.nan)
                    time_end.append(np.nan)
                    p_values.append(np.nan)
                else:
                    for clust in range(0, len(clust_idx)):
                        cluster = clust_idx[clust][0] #to get the slice sequence we need
        
                        cluster_start = cluster.start
                        clust_idx_start.append(cluster_start)
        
                        cluster_end = cluster.stop
                        clust_idx_end.append(cluster_end)
        
                        time_idx_start = time[cluster_start]
                        time_start.append(time_idx_start)
        
                        time_idx_end = time[cluster_end - 1] #minus one because python indexing does not include ending value
                        time_end.append(time_idx_end)
        
                        clust_p = clust_pvals[clust]
                        p_values.append(clust_p)
        
                        condition.append(theta_conditionnames[c])
            
            elif c == 1:
                T_0, clust_idx, clust_pvals, H0 = get_clust_perm_test(theta_medfro_rot_diff, theta_medfro_rdm_diff, p, perms)
                if len(clust_idx) == 0:
                    condition.append(theta_conditionnames[c])
                    clust_idx_start.append(np.nan)
                    clust_idx_end.append(np.nan)
                    time_start.append(np.nan)
                    time_end.append(np.nan)
                    p_values.append(np.nan)
                else:    
                    for clust in range(0, len(clust_idx)):
                        cluster = clust_idx[clust][0] #to get the slice sequence we need
        
                        cluster_start = cluster.start
                        clust_idx_start.append(cluster_start)
        
                        cluster_end = cluster.stop
                        clust_idx_end.append(cluster_end)
        
                        time_idx_start = time[cluster_start]
                        time_start.append(time_idx_start)
        
                        time_idx_end = time[cluster_end - 1] #minus one because python indexing does not include ending value
                        time_end.append(time_idx_end)
        
                        clust_p = clust_pvals[clust]
                        p_values.append(clust_p)
        
                        condition.append(theta_conditionnames[c])
            
            elif c == 2:
                T_0, clust_idx, clust_pvals, H0 = get_clust_perm_test(theta_medfro_mir_diff, theta_medfro_rdm_diff, p, perms)
                if len(clust_idx) == 0:
                    condition.append(theta_conditionnames[c])
                    clust_idx_start.append(np.nan)
                    clust_idx_end.append(np.nan)
                    time_start.append(np.nan)
                    time_end.append(np.nan)
                    p_values.append(np.nan)
                else:    
                    for clust in range(0, len(clust_idx)):
                        cluster = clust_idx[clust][0] #to get the slice sequence we need
        
                        cluster_start = cluster.start
                        clust_idx_start.append(cluster_start)
        
                        cluster_end = cluster.stop
                        clust_idx_end.append(cluster_end)
        
                        time_idx_start = time[cluster_start]
                        time_start.append(time_idx_start)
        
                        time_idx_end = time[cluster_end - 1] #minus one because python indexing does not include ending value
                        time_end.append(time_idx_end)
        
                        clust_p = clust_pvals[clust]
                        p_values.append(clust_p)
        
                        condition.append(theta_conditionnames[c])
    elif band == 1:
        alpha_conditionnames = ['alpha_medfro_rotvmir', 'alpha_medfro_rotvrdm', 'alpha_medfro_mirvrdm']
        for c in range(0, len(alpha_conditionnames)):
            if c == 0:
                T_0, clust_idx, clust_pvals, H0 = get_clust_perm_test(alpha_medfro_rot_diff, alpha_medfro_mir_diff, p, perms)
        #         print(clust_idx, clust_pvals)
                if len(clust_idx) == 0:
                    condition.append(alpha_conditionnames[c])
                    clust_idx_start.append(np.nan)
                    clust_idx_end.append(np.nan)
                    time_start.append(np.nan)
                    time_end.append(np.nan)
                    p_values.append(np.nan)
                else:
                    for clust in range(0, len(clust_idx)):
                        cluster = clust_idx[clust][0] #to get the slice sequence we need
        
                        cluster_start = cluster.start
                        clust_idx_start.append(cluster_start)
        
                        cluster_end = cluster.stop
                        clust_idx_end.append(cluster_end)
        
                        time_idx_start = time[cluster_start]
                        time_start.append(time_idx_start)
        
                        time_idx_end = time[cluster_end - 1] #minus one because python indexing does not include ending value
                        time_end.append(time_idx_end)
        
                        clust_p = clust_pvals[clust]
                        p_values.append(clust_p)
        
                        condition.append(alpha_conditionnames[c])
            
            elif c == 1:
                T_0, clust_idx, clust_pvals, H0 = get_clust_perm_test(alpha_medfro_rot_diff, alpha_medfro_rdm_diff, p, perms)
                if len(clust_idx) == 0:
                    condition.append(alpha_conditionnames[c])
                    clust_idx_start.append(np.nan)
                    clust_idx_end.append(np.nan)
                    time_start.append(np.nan)
                    time_end.append(np.nan)
                    p_values.append(np.nan)
                else:    
                    for clust in range(0, len(clust_idx)):
                        cluster = clust_idx[clust][0] #to get the slice sequence we need
        
                        cluster_start = cluster.start
                        clust_idx_start.append(cluster_start)
        
                        cluster_end = cluster.stop
                        clust_idx_end.append(cluster_end)
        
                        time_idx_start = time[cluster_start]
                        time_start.append(time_idx_start)
        
                        time_idx_end = time[cluster_end - 1] #minus one because python indexing does not include ending value
                        time_end.append(time_idx_end)
        
                        clust_p = clust_pvals[clust]
                        p_values.append(clust_p)
        
                        condition.append(alpha_conditionnames[c])
            
            elif c == 2:
                T_0, clust_idx, clust_pvals, H0 = get_clust_perm_test(alpha_medfro_mir_diff, alpha_medfro_rdm_diff, p, perms)
                if len(clust_idx) == 0:
                    condition.append(alpha_conditionnames[c])
                    clust_idx_start.append(np.nan)
                    clust_idx_end.append(np.nan)
                    time_start.append(np.nan)
                    time_end.append(np.nan)
                    p_values.append(np.nan)
                else:    
                    for clust in range(0, len(clust_idx)):
                        cluster = clust_idx[clust][0] #to get the slice sequence we need
        
                        cluster_start = cluster.start
                        clust_idx_start.append(cluster_start)
        
                        cluster_end = cluster.stop
                        clust_idx_end.append(cluster_end)
        
                        time_idx_start = time[cluster_start]
                        time_start.append(time_idx_start)
        
                        time_idx_end = time[cluster_end - 1] #minus one because python indexing does not include ending value
                        time_end.append(time_idx_end)
        
                        clust_p = clust_pvals[clust]
                        p_values.append(clust_p)
        
                        condition.append(alpha_conditionnames[c])
            
    elif band == 2:
        beta_conditionnames = ['beta_medfro_rotvmir', 'beta_medfro_rotvrdm', 'beta_medfro_mirvrdm']
        for c in range(0, len(beta_conditionnames)):
            if c == 0:
                T_0, clust_idx, clust_pvals, H0 = get_clust_perm_test(beta_medfro_rot_diff, beta_medfro_mir_diff, p, perms)
        #         print(clust_idx, clust_pvals)
                if len(clust_idx) == 0:
                    condition.append(beta_conditionnames[c])
                    clust_idx_start.append(np.nan)
                    clust_idx_end.append(np.nan)
                    time_start.append(np.nan)
                    time_end.append(np.nan)
                    p_values.append(np.nan)
                else:
                    for clust in range(0, len(clust_idx)):
                        cluster = clust_idx[clust][0] #to get the slice sequence we need
        
                        cluster_start = cluster.start
                        clust_idx_start.append(cluster_start)
        
                        cluster_end = cluster.stop
                        clust_idx_end.append(cluster_end)
        
                        time_idx_start = time[cluster_start]
                        time_start.append(time_idx_start)
        
                        time_idx_end = time[cluster_end - 1] #minus one because python indexing does not include ending value
                        time_end.append(time_idx_end)
        
                        clust_p = clust_pvals[clust]
                        p_values.append(clust_p)
        
                        condition.append(beta_conditionnames[c])
            
            elif c == 1:
                T_0, clust_idx, clust_pvals, H0 = get_clust_perm_test(beta_medfro_rot_diff, beta_medfro_rdm_diff, p, perms)
                if len(clust_idx) == 0:
                    condition.append(beta_conditionnames[c])
                    clust_idx_start.append(np.nan)
                    clust_idx_end.append(np.nan)
                    time_start.append(np.nan)
                    time_end.append(np.nan)
                    p_values.append(np.nan)
                else:    
                    for clust in range(0, len(clust_idx)):
                        cluster = clust_idx[clust][0] #to get the slice sequence we need
        
                        cluster_start = cluster.start
                        clust_idx_start.append(cluster_start)
        
                        cluster_end = cluster.stop
                        clust_idx_end.append(cluster_end)
        
                        time_idx_start = time[cluster_start]
                        time_start.append(time_idx_start)
        
                        time_idx_end = time[cluster_end - 1] #minus one because python indexing does not include ending value
                        time_end.append(time_idx_end)
        
                        clust_p = clust_pvals[clust]
                        p_values.append(clust_p)
        
                        condition.append(beta_conditionnames[c])
            
            elif c == 2:
                T_0, clust_idx, clust_pvals, H0 = get_clust_perm_test(beta_medfro_mir_diff, beta_medfro_rdm_diff, p, perms)
                if len(clust_idx) == 0:
                    condition.append(beta_conditionnames[c])
                    clust_idx_start.append(np.nan)
                    clust_idx_end.append(np.nan)
                    time_start.append(np.nan)
                    time_end.append(np.nan)
                    p_values.append(np.nan)
                else:    
                    for clust in range(0, len(clust_idx)):
                        cluster = clust_idx[clust][0] #to get the slice sequence we need
        
                        cluster_start = cluster.start
                        clust_idx_start.append(cluster_start)
        
                        cluster_end = cluster.stop
                        clust_idx_end.append(cluster_end)
        
                        time_idx_start = time[cluster_start]
                        time_start.append(time_idx_start)
        
                        time_idx_end = time[cluster_end - 1] #minus one because python indexing does not include ending value
                        time_end.append(time_idx_end)
        
                        clust_p = clust_pvals[clust]
                        p_values.append(clust_p)
        
                        condition.append(beta_conditionnames[c])
        
perm_test = pd.DataFrame(
    {'condition': condition,
     'clust_idx_start': clust_idx_start,
     'clust_idx_end': clust_idx_end,
     'time_start': time_start,
     'time_end': time_end,
     'p_values': p_values})

perm_test_filename = os.path.join('F:/Documents/Science/MirRevAdaptEEG/data/', 'TFR_Permutation_test_ErrorSize_PerturbTypeComp_%s_%s.csv' % (erps, roi))
perm_test.to_csv(perm_test_filename)